In [ ]:
import os
os.chdir("../../..")
import jax
jax.devices()

In [ ]:
from scripts.nj.neurosci import *
import scripts.nj.graph_to_arrays as ga
import scripts.data_preparation as dp
import networkx as nx

## Получаем данные

In [ ]:
neurons_ids = [
    "7055857",
    "1805418",
    "14260575",
    "5835799",
    "10160250",
    "7840203",
    "5019924",
    "13986477",
    "10167078",
    "7982896",
    "4119387",
    "17591442",
    "4227544",
    "10495502",
    "8069478",
    "3913629",
    "11279244",
    "16846805",
    "8980589",
    "3664102",
]

DIR = "Datasets/Generated/trash"
neurons_ids = [int(i) for i in neurons_ids]
sc = dp.simulation_context(DIR, neurons_ids)
sc.build_full_graph()
full_g = nx.read_gml(sc.path_to_full_graph)

In [ ]:
jc = ga.SimulationContextJax(graph = full_g, node_type_groups = {
    'cable':['branch', 'root', 'slab', 'end'],
    'alpha':['connector']
}, edge_directedness={'cable': {'cable': False},}, initial_node_values={
    'cable':1.0
}, cache_dir=DIR + '/jax')

In [ ]:
res = jc.get_context()

In [ ]:
res.keys()

In [ ]:
metadata = sc.node_metadata
metadata

In [ ]:
metadata = metadata.fillna(10.0) # 10.0 as basic radius
metadata['new_index'] = metadata.apply(lambda row:res['mapping']['cable'].get(str(row['node_id'])), axis = 1)
metadata = metadata.dropna(subset=['new_index'])
metadata = metadata.set_index('new_index').sort_index()
metadata

In [ ]:
all_somas = metadata[metadata['type'] == 'root']['node_id'].to_numpy()

In [ ]:
stom = [(int(soma), int(res['mapping']['cable'][str(soma)])) for soma in all_somas]
stom = jnp.array(stom)
stom

## Определяем структуру симуляции

In [ ]:
ind_to_stim = stom[0][1]
def get_my_pipeline(constants, dt = 0.1):
    r = jnp.array(metadata['radius'].to_numpy())
    x = jnp.array(metadata['x'].to_numpy())
    y = jnp.array(metadata['y'].to_numpy())
    z = jnp.array(metadata['z'].to_numpy())
    S = np.pi * r**2
    cable_m = res['edges_cable_to_cable'].T
    pre_syn = res['edges_cable_to_alpha'].T
    post_syn = res['edges_cable_to_alpha'].T

    dx = x.at[cable_m[:, 1]].get() - x.at[cable_m[:, 0]].get()
    dy = y.at[cable_m[:, 1]].get() - y.at[cable_m[:, 0]].get()
    dz = z.at[cable_m[:, 1]].get() - z.at[cable_m[:, 0]].get()

    L = (dx**2 + dy**2 + dz**2)**0.5
    ro = 1.0 # по идеи должно быть 100, но потом разберемся
    R = (ro*L/S.at[cable_m[:, 1]].get())

    HH = get_HH_pipeline(**constants) # получаем функцию для HH
    cable = laplace_at_graph_symetric(cable_m, 'V')#, scaling = R) # получаем функцию для динамики кабелей
    alphaP = get_alpha_synapce_pipeline(pre_syn, post_syn, **constants)
    
    @jax.jit
    def state_transformed(state):
        s, ds = to_diff(state) # создает ds той же формы что и state, но заполненный нулями
        s, ds = alphaP(s, ds)
        s, ds = HH(s, ds) # вставляет HH каналы
        s, ds = cable(s, ds) # соединяет сегменты
        ds['V'] += ds['V'].at[ind_to_stim].add(100.0)
        return s, ds

    integrate = get_runge_kutta_step(state_transformed, dt) # получаем функцию для интегрирования
    @jax.jit
    def my_pipeline(state):
        s = integrate(state) # интегрируем
        return s
    return my_pipeline


### Начальные значения и константы


In [ ]:
num_nodes_hh = res['num_nodes']['cable']
num_synapces = res['num_nodes']['alpha']
total_nodes = num_nodes_hh + num_synapces

initials = {
    "V":jnp.ones((num_nodes_hh, ), jnp.float32)*-65.0,
    "m":jnp.ones((num_nodes_hh, ), jnp.float32)*0.0220,
    'n':jnp.ones((num_nodes_hh, ), jnp.float32)*0.0773,
    'h':jnp.ones((num_nodes_hh, ), jnp.float32)*0.9840,
    'alpha':jnp.ones((num_nodes_hh, 2), jnp.float32)*0.1,
    "time":0.0
}

consts = {
    "C": jnp.ones((num_nodes_hh, ), jnp.float32),# Емкость мембраны (мкФ/см^2)
    "ENa": 50.0,   # Равновесный потенциал Na+ (мВ)
    "EK": -77.0,   # Равновесный потенциал K+ (мВ)
    "EL": -54.4,   # Равновесный потенциал утечки (мВ)
    "gNa": 120.0,  # Максимальная проводимость Na+ (мСм/см^2)
    "gK": 36.0,    # Максимальная проводимость K+ (мСм/см^2)
    "gL": 0.3,     # Проводимость утечки (мСм/см^2),
    "tau":1.0,
    'E_rev':1.0,
    'V_m':jnp.ones((num_nodes_hh, ), jnp.float32),
    'alpha_syn_detector_treshold':40.0,
    'synaptic_weights':0.01,
    'G_max':1.0
}


new_consts = {
    "C": jnp.ones((num_nodes_hh, ), jnp.float32),# Емкость мембраны (мкФ/см^2)
    "ENa": 50.0,   # Равновесный потенциал Na+ (мВ)
    "EK": -77.0,   # Равновесный потенциал K+ (мВ)
    "EL": -54.4,   # Равновесный потенциал утечки (мВ)
    "gNa": 0.12,  # Максимальная проводимость Na+ (мСм/см^2)
    "gK": 0.036,    # Максимальная проводимость K+ (мСм/см^2)
    "gL": 0.0003,     # Проводимость утечки (мСм/см^2),
    "tau":0.5,
    'E_rev':0.0,
    'V_m':jnp.ones((num_nodes_hh, ), jnp.float32),
    'alpha_syn_detector_treshold':0.5,
    'synaptic_weights':0.5,
    'G_max':1.0
}

In [ ]:
my_pipeline = get_my_pipeline(consts, 0.1)

## Запуск симуляции

In [ ]:

@jax.jit
def output_transform(state):
    return state['time'], state['V'].at[stom[:, 1]].get()


jsim = simulation(initials, my_pipeline, 100, output_transform)
H = jsim.run(100)

In [ ]:
H

## Построение графиков

In [ ]:
import matplotlib.pyplot as plt
t, v = H
t, v = np.array(t), np.array(v)
plt.plot(v)
plt.show()